In [27]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [28]:
# Store filepath in a variable.
state_file = "../Datasets/State_Zhvi_AllHomes.csv"
county_file = "../Datasets/County_Zhvi_AllHomes.csv"
zip_code_file = "../Datasets/Zip_Zhvi_AllHomes.csv"

In [29]:
# Read our Data file with the pandas library, and create dataframes.
# The Zillow CSV files require ISO-8859-1 encoding.
state_data = pd.read_csv(state_file, encoding="ISO-8859-1")        # Zillow Home Value Index (ZHVI)
county_data = pd.read_csv(county_file, encoding="ISO-8859-1")      # Zillow Home Value Index (ZHVI)
zip_code_data = pd.read_csv(zip_code_file, encoding="ISO-8859-1")  # Zillow Home Value Index (ZHVI)

In [30]:
# Cleaning up zip code data
# Place the rows with San Diego County into a dataframe
zip_sd_county_row = zip_code_data.loc[zip_code_data["CountyName"] == "San Diego County", :]
del zip_sd_county_row['RegionID']
del zip_sd_county_row['State']
del zip_sd_county_row['Metro']
del zip_sd_county_row['CountyName']
zip_sd_county_row

# Set the index to be region name
sd_county_zips_index = zip_sd_county_row.set_index("RegionName")
sd_county_zips_index.head()

,City,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,...,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01
RegionName,,,,,,,,,,,,,,,,,,,,,
91910,Chula Vista,119,163800.0,163500.0,163100.0,162700.0,162400.0,162300.0,162300.0,162500.0,...,525900,526500,526500,527500,531100,534200,535800,536500,536400,535700
92126,San Diego,183,153700.0,153200.0,152700.0,152100.0,151500.0,151000.0,150600.0,150300.0,...,588300,588100,589800,594200,598700,600600,602700,605800,607500,607100
91911,Chula Vista,197,143800.0,143500.0,143100.0,142700.0,142500.0,142300.0,142300.0,142400.0,...,474500,474200,474700,476600,479200,481100,483000,484900,486000,486200
92101,San Diego,230,147800.0,147800.0,147900.0,148100.0,148400.0,148800.0,149400.0,150000.0,...,594400,594500,594500,594200,594300,594700,595700,596400,595500,593800
92109,San Diego,231,209400.0,208400.0,207300.0,206200.0,205100.0,204000.0,203200.0,202700.0,...,943200,952100,958100,965000,974300,977400,974300,969400,964800,960300


In [31]:
# Find column index for desired date range start and end
start = sd_county_zips_index.columns.get_loc("2008-12")
end = sd_county_zips_index.columns.get_loc("2018-12")

# Make a new dataframe containing the 10 year range of data Dec 2008 - Dec 2018
# range_dates = sd_county_zips_index.iloc[:, start:end+1]
sd_county_zips_10yr_city_rank = pd.DataFrame(sd_county_zips_index, columns=["City", "SizeRank"])

sd_county_zips_10yr_date_range = sd_county_zips_index.iloc[:, start:end+1]

sd_county_zips_10yr = pd.merge(sd_county_zips_10yr_city_rank, sd_county_zips_10yr_date_range, on="RegionName")
sd_county_zips_10yr

,City,SizeRank,2008-12,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
RegionName,,,,,,,,,,,,,,,,,,,,,
91910,Chula Vista,119,348100.0,342100.0,336200.0,329600.0,322400.0,314800.0,308500.0,304500.0,...,524100,525900,526500,526500,527500,531100,534200,535800,536500,536400
92126,San Diego,183,371900.0,370000.0,367600.0,364300.0,360400.0,356100.0,353500.0,353800.0,...,587400,588300,588100,589800,594200,598700,600600,602700,605800,607500
91911,Chula Vista,197,310300.0,304500.0,298700.0,292300.0,285400.0,278200.0,272100.0,268400.0,...,474300,474500,474200,474700,476600,479200,481100,483000,484900,486000
92101,San Diego,230,450000.0,447500.0,444100.0,439800.0,434800.0,430000.0,425300.0,421600.0,...,594000,594400,594500,594500,594200,594300,594700,595700,596400,595500
92109,San Diego,231,616500.0,616800.0,615800.0,613000.0,609600.0,606200.0,605200.0,608300.0,...,935800,943200,952100,958100,965000,974300,977400,974300,969400,964800
92021,El Cajon,257,334000.0,330400.0,326200.0,321000.0,315200.0,309500.0,305100.0,303000.0,...,479100,479500,478900,477900,478100,479900,480700,481500,483100,483900
92154,San Diego,445,303600.0,296900.0,290000.0,282200.0,273900.0,265400.0,258500.0,254400.0,...,460400,462300,463300,464600,467300,470900,473100,475000,477400,479000
92104,San Diego,446,364600.0,362600.0,360000.0,356500.0,352200.0,347800.0,344300.0,342300.0,...,616600,616600,614700,612300,612400,615600,617600,617500,616900,616800
92115,San Diego,453,339200.0,336400.0,333100.0,329200.0,324400.0,319400.0,315800.0,314800.0,...,527800,527400,526300,526100,526900,529000,531100,533600,536100,537200


In [32]:
# Make a dataframe for summary
summary_zips_base = pd.DataFrame(sd_county_zips_10yr, columns=["2008-12", "2018-12"])
summary_zips = summary_zips_base.rename(columns={"2008-12": "2008-Q4 Home Value", "2018-12": "2018-Q4 Home Value"})
summary_zips.index.names = ['Zip Code']
summary_zips["10-Yr Home Value Change"] = (summary_zips["2018-Q4 Home Value"] - summary_zips["2008-Q4 Home Value"])*100 / summary_zips["2008-Q4 Home Value"]

# Make a copy of the summary dataframe that is formatted for display
summary_zips_formatted = summary_zips.copy()
summary_zips_formatted["2008-Q4 Home Value"] = summary_zips_formatted["2008-Q4 Home Value"].map("${:,.0f}".format)
summary_zips_formatted["2018-Q4 Home Value"] = summary_zips_formatted["2018-Q4 Home Value"].map("${:,.0f}".format)
summary_zips_formatted["10-Yr Home Value Change"] = summary_zips_formatted["10-Yr Home Value Change"].map("{:.2f}%".format)
summary_zips_formatted.head()

,2008-Q4 Home Value,2018-Q4 Home Value,10-Yr Home Value Change
Zip Code,,,
91910,"$348,100","$536,400",54.09%
92126,"$371,900","$607,500",63.35%
91911,"$310,300","$486,000",56.62%
92101,"$450,000","$595,500",32.33%
92109,"$616,500","$964,800",56.50%


In [34]:
# Rank the 10 year zip code data by value change (%) descending
ranked_zips = summary_zips.sort_values(["10-Yr Home Value Change"], ascending=False)
ranked_zips.head()

# Make a copy of the sorted dataframe that is formatted for display
ranked_zips_formatted = ranked_zips.copy()
ranked_zips_formatted["2008-Q4 Home Value"] = ranked_zips_formatted["2008-Q4 Home Value"].map("${:,.0f}".format)
ranked_zips_formatted["2018-Q4 Home Value"] = ranked_zips_formatted["2018-Q4 Home Value"].map("${:,.0f}".format)
ranked_zips_formatted["10-Yr Home Value Change"] = ranked_zips_formatted["10-Yr Home Value Change"].map("{:.2f}%".format)
ranked_zips_formatted.head()

,2008-Q4 Home Value,2018-Q4 Home Value,10-Yr Home Value Change
Zip Code,,,
92007,"$718,400","$1,300,400",81.01%
91932,"$337,200","$605,500",79.57%
92116,"$366,700","$641,300",74.88%
92113,"$235,900","$406,300",72.23%
92104,"$364,600","$616,800",69.17%


In [ ]:
# # Place the row with CA into a variable
# ca_row = state_data.loc[state_data["RegionName"] == "California", :]

# # Place the row with SD County into a variable
# sd_row = county_data.loc[county_data["RegionName"] == "San Diego County", :]
# del sd_row['State']
# del sd_row['Metro']
# del sd_row['StateCodeFIPS']
# del sd_row['MunicipalCodeFIPS']

# # Place the rows with **highest appreciation rate** into a variable
# zip_high_row = zip_code_data.loc[zip_code_data["RegionName"] == 92037, :]
# del zip_92037_row['City']
# del zip_92037_row['State']
# del zip_92037_row['Metro']
# del zip_92037_row['CountyName']

# # Place the rows with **lowest appreciation rate** into a variable
# zip_low_row = zip_code_data.loc[zip_code_data["RegionName"] == 92071, :]
# del zip_92071_row['City']
# del zip_92071_row['State']
# del zip_92071_row['Metro']
# del zip_92071_row['CountyName']

# ----------------------------------------------------------------------------------------------

# # Get list of column names
# column_names = list(state_data.columns.values)
# # Create empty dataframe
# state_county_zip = pd.DataFrame(columns=column_names)

# # Add our data rows to the empty dataframe 
# state_county_zip = state_county_zip.append(ca_row)
# state_county_zip = state_county_zip.append(sd_row)
# state_county_zip = state_county_zip.append(zip_92037_row)
# state_county_zip = state_county_zip.append(zip_92071_row)

# # # Set the index to be region name
# state_county_zip_index = state_county_zip.set_index("RegionName")
# state_county_zip_index.head()

# # Show DF
# state_county_zip_index.head()

# ----------------------------------------------------------------------------------------------

# # Find column index for desired date range start and end
# start = state_county_zip_index.columns.get_loc("2008-12")
# end = state_county_zip_index.columns.get_loc("2018-12")

# # Make a new dataframe containing the 10 year range of data Dec 2008 - Dec 2018
# state_county_zip_10yr = state_county_zip_index.iloc[:, start:end+1]
# state_county_zip_10yr